In [3]:
import cv2
import numpy as np
import time
from collections import deque, defaultdict
from dataclasses import dataclass
from typing import List, Tuple, Dict

In [4]:
print("=" * 80)
print("F1 RACING ANALYTICS - STANDALONE DEMO".center(80))
print("=" * 80)

                     F1 RACING ANALYTICS - STANDALONE DEMO                      


In [ ]:
@dataclass
class Vehicle:
    id: int  #driver-id
    driver: str #driver name 
    team: str        #drivers team name 
    color: Tuple[int, int, int]
    position: int   #position of the driver
    track_pos: float #position of the driver on the track
    speed: float  #current speed of the driver 
    lap_count: int #lap counter for the driver 
    current_lap_time: float #exsisting lap time of the driver
    best_lap_time: float #hot lap timings of the driver - PURPLE SECTOR
    last_lap_time: float #recent lap time of the driver 
    x: int
    y: int
    history: deque

In [11]:
class F1Simulator:
    """Simulates F1 race for demonstration"""
    
    def __init__(self, width=1280, height=720):
        self.width = width
        self.height = height
        self.track_center_x = width // 2
        self.track_center_y = height // 2
        self.track_radius_x = 450
        self.track_radius_y = 250
        
        # F1 2024 Grid
        self.drivers = [
            ("Verstappen", "Red Bull", (220, 0, 0)),
            ("Hamilton", "Mercedes", (0, 210, 190)),
            ("Leclerc", "Ferrari", (220, 0, 0)),
            ("Norris", "McLaren", (255, 135, 0)),
            ("Sainz", "Ferrari", (220, 0, 0)),
            ("Perez", "Red Bull", (30, 65, 255)),
            ("Russell", "Mercedes", (0, 210, 190)),
            ("Alonso", "Aston Martin", (0, 111, 98)),
        ]
        
        self.vehicles = []
        self.frame_count = 0
        self.race_status = "Formation Lap"
        self.fastest_lap = {"driver": None, "time": float('inf')}
        self.overtakes = 0
        
        # Initialize vehicles
        for i, (driver, team, color) in enumerate(self.drivers):
            vehicle = Vehicle(
                id=i,
                driver=driver,
                team=team,
                color=color,
                position=i + 1,
                track_pos=(i * 0.11) % 1.0,
                speed=290 + np.random.randint(-10, 20),
                lap_count=0,
                current_lap_time=0.0,
                best_lap_time=float('inf'),
                last_lap_time=0.0,
                x=0,
                y=0,
                history=deque(maxlen=50)
            )
            self.vehicles.append(vehicle)
    
    def update(self):
        """Update simulation state"""
        self.frame_count += 1
        
        # Change to green flag after 5 seconds (150 frames @ 30fps)
        if self.frame_count == 150:
            self.race_status = "🟢 GREEN FLAG"
            print("\n🏁 LIGHTS OUT AND AWAY WE GO!\n")
        
        # Update each vehicle
        for vehicle in self.vehicles:
            # Update track position
            speed_factor = (vehicle.speed / 300) * 0.004
            
            # Add some racing variation
            variation = np.sin(self.frame_count * 0.05 + vehicle.id * 0.5) * 0.0003
            
            old_pos = vehicle.track_pos
            vehicle.track_pos = (vehicle.track_pos + speed_factor + variation) % 1.0
            
            # Update lap timing
            vehicle.current_lap_time += 1/30  # 30 fps
            
            # Check for lap completion
            if vehicle.track_pos < old_pos and vehicle.current_lap_time > 30:
                vehicle.lap_count += 1
                vehicle.last_lap_time = vehicle.current_lap_time
                
                # Update best lap
                if vehicle.current_lap_time < vehicle.best_lap_time:
                    vehicle.best_lap_time = vehicle.current_lap_time
                    
                    # Update fastest lap
                    if vehicle.current_lap_time < self.fastest_lap['time']:
                        self.fastest_lap = {
                            'driver': vehicle.driver,
                            'time': vehicle.current_lap_time
                        }
                        print(f"⚡ FASTEST LAP: {vehicle.driver} - {self._format_time(vehicle.current_lap_time)}")
                
                vehicle.current_lap_time = 0.0
            
            # Calculate screen position
            angle = vehicle.track_pos * 2 * np.pi
            vehicle.x = int(self.track_center_x + self.track_radius_x * np.cos(angle))
            vehicle.y = int(self.track_center_y + self.track_radius_y * np.sin(angle))
            vehicle.history.append((vehicle.x, vehicle.y))
            
            # Speed variation
            vehicle.speed += np.random.uniform(-2, 2)
            vehicle.speed = np.clip(vehicle.speed, 200, 340)
        
        # Update positions
        self.vehicles.sort(key=lambda v: (-v.lap_count, -v.track_pos))
        for i, vehicle in enumerate(self.vehicles):
            old_position = vehicle.position
            vehicle.position = i + 1
            
            # Detect overtakes
            if old_position > vehicle.position and self.frame_count > 150:
                self.overtakes += 1
                print(f"🔥 OVERTAKE! {vehicle.driver} moves to P{vehicle.position}")
            
            # Calculate gaps
            if i == 0:
                vehicle.gap_to_leader = 0.0
            else:
                leader = self.vehicles[0]
                vehicle.gap_to_leader = (leader.lap_count - vehicle.lap_count) * 85 + \
                                       (leader.track_pos - vehicle.track_pos) * 85
    
    def draw_frame(self):
        """Draw current frame"""
        frame = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        
        # Draw track
        self._draw_track(frame)
        
        # Draw vehicles
        for vehicle in self.vehicles:
            self._draw_vehicle(frame, vehicle)
        
        # Draw HUD
        self._draw_hud(frame)
        
        return frame
    
    def _draw_track(self, frame):
        """Draw race track"""
        # Main track oval
        cv2.ellipse(frame, 
                   (self.track_center_x, self.track_center_y),
                   (self.track_radius_x, self.track_radius_y),
                   0, 0, 360, (100, 100, 100), 3)
        
        # Track edges
        cv2.ellipse(frame,
                   (self.track_center_x, self.track_center_y),
                   (self.track_radius_x + 40, self.track_radius_y + 40),
                   0, 0, 360, (60, 60, 60), 2)
        
        cv2.ellipse(frame,
                   (self.track_center_x, self.track_center_y),
                   (self.track_radius_x - 40, self.track_radius_y - 40),
                   0, 0, 360, (60, 60, 60), 2)
        
        # Start/Finish line
        start_x = self.track_center_x + self.track_radius_x
        cv2.line(frame, (start_x, self.track_center_y - 50),
                (start_x, self.track_center_y + 50), (255, 255, 255), 3)
        
        # Checkered pattern
        for i in range(10):
            color = (255, 255, 255) if i % 2 == 0 else (0, 0, 0)
            cv2.rectangle(frame,
                         (start_x - 5, self.track_center_y - 50 + i * 10),
                         (start_x + 5, self.track_center_y - 50 + (i + 1) * 10),
                         color, -1)
    
    def _draw_vehicle(self, frame, vehicle):
        """Draw individual vehicle"""
        x, y = vehicle.x, vehicle.y
        
        # Racing line (trail)
        if len(vehicle.history) > 1:
            points = np.array(list(vehicle.history), dtype=np.int32)
            cv2.polylines(frame, [points], False, vehicle.color, 2)
        
        # Car body
        size = 25 if vehicle.position <= 3 else 20
        cv2.circle(frame, (x, y), size, vehicle.color, -1)
        cv2.circle(frame, (x, y), size + 2, (255, 255, 255), 2)
        
        # Position number
        cv2.putText(frame, str(vehicle.position), (x - 8, y + 8),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Driver name and speed
        label = f"{vehicle.driver}"
        cv2.putText(frame, label, (x - 40, y - 35),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, vehicle.color, 2)
        
        speed_label = f"{int(vehicle.speed)} km/h"
        cv2.putText(frame, speed_label, (x - 35, y - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    def _draw_hud(self, frame):
        """Draw F1-style HUD"""
        # Race status banner
        cv2.rectangle(frame, (self.width // 2 - 200, 20),
                     (self.width // 2 + 200, 70), (0, 0, 0), -1)
        cv2.rectangle(frame, (self.width // 2 - 200, 20),
                     (self.width // 2 + 200, 70), (255, 255, 255), 2)
        
        status_color = (0, 255, 0) if "GREEN" in self.race_status else (255, 255, 0)
        cv2.putText(frame, self.race_status, (self.width // 2 - 180, 52),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, status_color, 2)
        
        # Timing tower
        self._draw_timing_tower(frame)
        
        # Mini map
        self._draw_minimap(frame)
        
        # Stats panel
        self._draw_stats(frame)
        
        # Telemetry for leader
        if self.vehicles:
            self._draw_telemetry(frame, self.vehicles[0])
    
    def _draw_timing_tower(self, frame):
        """Draw timing tower"""
        x, y = 20, 100
        
        # Background
        cv2.rectangle(frame, (x, y), (x + 300, y + 350), (20, 20, 20), -1)
        cv2.rectangle(frame, (x, y), (x + 300, y + 350), (255, 255, 255), 2)
        
        # Header
        cv2.putText(frame, "LIVE TIMING", (x + 15, y + 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Drivers
        y_offset = y + 60
        for i, vehicle in enumerate(self.vehicles[:8]):
            # Position indicator
            cv2.circle(frame, (x + 20, y_offset - 5), 8, vehicle.color, -1)
            
            # Position number
            cv2.putText(frame, f"P{vehicle.position}", (x + 40, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Driver name
            cv2.putText(frame, vehicle.driver[:10], (x + 85, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
            
            # Gap
            if vehicle.position == 1:
                gap_text = "Leader"
            else:
                gap_text = f"+{vehicle.gap_to_leader:.1f}s"
            
            cv2.putText(frame, gap_text, (x + 200, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
            
            y_offset += 35
    
    def _draw_minimap(self, frame):
        """Draw mini track map"""
        x, y = self.width - 220, 20
        size = 180
        
        # Background
        cv2.rectangle(frame, (x, y), (x + size, y + size), (20, 20, 20), -1)
        cv2.rectangle(frame, (x, y), (x + size, y + size), (255, 255, 255), 2)
        
        # Track
        center_x, center_y = x + size // 2, y + size // 2
        cv2.ellipse(frame, (center_x, center_y), (70, 50), 0, 0, 360, (100, 100, 100), 2)
        
        # Vehicles on map
        for vehicle in self.vehicles:
            angle = vehicle.track_pos * 2 * np.pi
            map_x = int(center_x + 70 * np.cos(angle))
            map_y = int(center_y + 50 * np.sin(angle))
            
            dot_size = 6 if vehicle.position <= 3 else 4
            cv2.circle(frame, (map_x, map_y), dot_size, vehicle.color, -1)
    
    def _draw_stats(self, frame):
        """Draw statistics panel"""
        x, y = 20, 470
        
        cv2.rectangle(frame, (x, y), (x + 300, y + 120), (20, 20, 20), -1)
        cv2.rectangle(frame, (x, y), (x + 300, y + 120), (255, 255, 255), 2)
        
        cv2.putText(frame, "RACE STATISTICS", (x + 15, y + 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Fastest lap
        if self.fastest_lap['driver']:
            fl_text = f"Fastest: {self.fastest_lap['driver']} - {self._format_time(self.fastest_lap['time'])}"
            cv2.putText(frame, fl_text, (x + 15, y + 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1)
        
        # Overtakes
        cv2.putText(frame, f"Overtakes: {self.overtakes}", (x + 15, y + 85),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 0), 1)
        
        # Frame counter
        cv2.putText(frame, f"Frame: {self.frame_count}", (x + 15, y + 105),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
    
    def _draw_telemetry(self, frame, vehicle):
        """Draw telemetry for leader"""
        x, y = self.width - 370, self.height - 170
        
        cv2.rectangle(frame, (x, y), (x + 350, y + 150), (20, 20, 20), -1)
        cv2.rectangle(frame, (x, y), (x + 350, y + 150), (255, 255, 255), 2)
        
        cv2.putText(frame, f"LEADER: {vehicle.driver}", (x + 15, y + 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, vehicle.color, 2)
        
        y_offset = y + 60
        cv2.putText(frame, f"Lap: {vehicle.lap_count}", (x + 15, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        cv2.putText(frame, f"Speed: {int(vehicle.speed)} km/h", (x + 180, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        y_offset += 30
        cv2.putText(frame, f"Current: {self._format_time(vehicle.current_lap_time)}",
                   (x + 15, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
        
        y_offset += 25
        cv2.putText(frame, f"Best: {self._format_time(vehicle.best_lap_time)}",
                   (x + 15, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1)
        
        if vehicle.last_lap_time > 0:
            cv2.putText(frame, f"Last: {self._format_time(vehicle.last_lap_time)}",
                       (x + 180, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (200, 200, 200), 1)
    
    def _format_time(self, seconds):
        """Format time"""
        if seconds == float('inf') or seconds == 0:
            return "--:--.---"
        minutes = int(seconds // 60)
        secs = seconds % 60
        return f"{minutes:01d}:{secs:06.3f}"

In [10]:
def run_f1_demo(duration_seconds=60, save_video=False):
    """
    Run F1 Racing Analytics Demo
    
    Args:
        duration_seconds: How long to run the demo (default 60s)
        save_video: Save output to video file (default False)
    """
    
    print("\n🏎️  Initializing F1 Race Simulation...")
    print("=" * 80)
    
    # Create simulator
    simulator = F1Simulator(width=1280, height=720)
    
    # Video writer setup
    writer = None
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter('f1_demo_output.mp4', fourcc, 30, (1280, 720))
        print("💾 Recording to: f1_demo_output.mp4")
    
    print("\n🏁 Starting Race Simulation...")
    print("   Press 'Q' to quit")
    print("   Press 'SPACE' to pause/resume")
    print("=" * 80 + "\n")
    
    fps = 30
    max_frames = duration_seconds * fps
    frame_count = 0
    paused = False
    
    start_time = time.time()
    
    try:
        while frame_count < max_frames:
            if not paused:
                # Update simulation
                simulator.update()
                
                # Draw frame
                frame = simulator.draw_frame()
                
                # Save if recording
                if writer:
                    writer.write(frame)
                
                frame_count += 1
            else:
                frame = simulator.draw_frame()
            
            # Display
            cv2.imshow('F1 Racing Analytics - Live Demo', frame)
            
            # Handle keyboard
            key = cv2.waitKey(int(1000/fps)) & 0xFF
            if key == ord('q') or key == 27:  # Q or ESC
                print("\n⏹️  Demo stopped by user")
                break
            elif key == ord(' '):  # SPACE
                paused = not paused
                print("⏸️  PAUSED" if paused else "▶️  RESUMED")
        
        elapsed = time.time() - start_time
        
        print("\n" + "=" * 80)
        print("🏁 RACE COMPLETE!")
        print("=" * 80)
        print(f"⏱️  Duration: {elapsed:.2f}s")
        print(f"📊 Frames processed: {frame_count}")
        print(f"⚡ Average FPS: {frame_count/elapsed:.1f}")
        
        if simulator.fastest_lap['driver']:
            print(f"\n🏆 Fastest Lap: {simulator.fastest_lap['driver']} - "
                  f"{simulator._format_time(simulator.fastest_lap['time'])}")
        
        print(f"🔥 Total Overtakes: {simulator.overtakes}")
        
        print("\n🏁 FINAL STANDINGS:")
        print(f"{'Pos':<5} {'Driver':<15} {'Team':<15} {'Laps':<6} {'Best Lap':<12}")
        print("-" * 65)
        
        for vehicle in simulator.vehicles[:8]:
            best_lap = simulator._format_time(vehicle.best_lap_time)
            print(f"{vehicle.position:<5} {vehicle.driver:<15} {vehicle.team:<15} "
                  f"{vehicle.lap_count:<6} {best_lap:<12}")
        
        print("=" * 80)
        
    except KeyboardInterrupt:
        print("\n⏹️  Demo interrupted")
    
    finally:
        # Cleanup
        if writer:
            writer.release()
            print("\n✅ Video saved to: f1_demo_output.mp4")
        
        cv2.destroyAllWindows()
        print("\n👋 Demo complete!")

In [12]:
if __name__ == "__main__":
    print("\n🏁 F1 RACING ANALYTICS - STANDALONE DEMO")
    print("=" * 80)
    print("\nThis demo simulates an F1 race with:")
    print("  ✅ Real-time vehicle tracking")
    print("  ✅ Live timing tower")
    print("  ✅ Speed telemetry")
    print("  ✅ Lap timing and fastest laps")
    print("  ✅ Position changes and overtakes")
    print("  ✅ Mini track map")
    print("  ✅ F1-style HUD")
    print("\n" + "=" * 80)
    
    # Configuration
    DEMO_DURATION = 60  # seconds
    SAVE_VIDEO = False  # Set to True to save output
    
    input("\nPress ENTER to start the demo...")
    
    # Run the demo
    run_f1_demo(duration_seconds=DEMO_DURATION, save_video=SAVE_VIDEO)
    
    print("\n✅ To run again: python f1_analytics_standalone_demo.py")
    print("💡 To save video: Set SAVE_VIDEO = True in the script")



🏁 F1 RACING ANALYTICS - STANDALONE DEMO

This demo simulates an F1 race with:
  ✅ Real-time vehicle tracking
  ✅ Live timing tower
  ✅ Speed telemetry
  ✅ Lap timing and fastest laps
  ✅ Position changes and overtakes
  ✅ Mini track map
  ✅ F1-style HUD


🏎️  Initializing F1 Race Simulation...

🏁 Starting Race Simulation...
   Press 'Q' to quit
   Press 'SPACE' to pause/resume


🏁 LIGHTS OUT AND AWAY WE GO!

🔥 OVERTAKE! Leclerc moves to P1
🔥 OVERTAKE! Hamilton moves to P2
🔥 OVERTAKE! Verstappen moves to P3
🔥 OVERTAKE! Alonso moves to P4
🔥 OVERTAKE! Russell moves to P5
🔥 OVERTAKE! Perez moves to P6
🔥 OVERTAKE! Sainz moves to P7
🔥 OVERTAKE! Russell moves to P4
🔥 OVERTAKE! Hamilton moves to P1
🔥 OVERTAKE! Verstappen moves to P2
🔥 OVERTAKE! Russell moves to P3
🔥 OVERTAKE! Alonso moves to P4
🔥 OVERTAKE! Perez moves to P5
🔥 OVERTAKE! Sainz moves to P6
🔥 OVERTAKE! Norris moves to P7
🔥 OVERTAKE! Verstappen moves to P1
🔥 OVERTAKE! Russell moves to P2
🔥 OVERTAKE! Alonso moves to P3
🔥 OVERTAKE! 